In [1]:
import numpy as np
import pandas as pd
import os
import torch

os.chdir("../")

from src.conf import LAYERS_DIMS, MODEL_FEATURES
from src.utils import get_device
from src.data import DataLoaders
from src.models import FraudAutoEncoder
from src.models import Model_Inference

ImportError: cannot import name 'Dataframe' from 'pandas' (/opt/mamba/lib/python3.10/site-packages/pandas/__init__.py)

## Data Gen

In [ ]:
n_, k_ = 1_000, 300

In [ ]:
pd.DataFrame(
    data=np.random.uniform(** {"low": -1, "high": 1, "size": [n_, k_]}),
    columns=[f"X_{str(i).zfill(3)}" for i in range(1, 1+k_)]
).to_parquet("./data/simulated_raw_data_new_arrival.gzip", compression="gzip")

In [ ]:
X = pd.read_parquet("/home/onyxia/work/fraud_detection/data/simulated_raw_data_new_arrival.gzip")

## Model Loading

In [ ]:
Model_Inference("simulated_data")._error_eval(X.values[98])

In [ ]:
!nvidia-smi

In [ ]:
import s3fs
import yaml

with open('./data/acces-key.yaml', 'r') as file:
    access_keys = yaml.safe_load(file)

fs = s3fs.S3FileSystem(
    key = access_keys["key"],
    secret = access_keys["secret"]
)

pd.read_parquet(
    fs.open(fs.ls('customer-data-platform-retail/test_data')[0])
)

In [ ]:
fs.ls('customer-data-platform-retail/test_data')